In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.3 MB/s eta 0:00:00


In [6]:
# ==============================================================================
# 1. CÀI ĐẶT VÀ NHẬP THƯ VIỆN MẠNH (XGBoost, LightGBM, CatBoost)
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import VotingClassifier

# Import các thuật toán Boosting mạnh mẽ
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# 2. ĐỌC DỮ LIỆU
# ==============================================================================
train_df = pd.read_csv('train (1).csv')
test_df = pd.read_csv('test (2).csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# ==============================================================================
# 3. FEATURE ENGINEERING & PREPROCESSING (XỬ LÝ DỮ LIỆU NÂNG CAO)
# ==============================================================================

# 3.1 Xử lý cột Text
# Thay vì bỏ 'Track Name', ta có thể bỏ nó để tránh nhiễu (Overfitting)
# vì Track Name quá unique. Tuy nhiên, Artist Name rất quan trọng.
cols_to_drop = ['Track Name']
# Nếu trong data có cột Id thì bỏ luôn, nếu không thì giữ nguyên logic
if 'Id' in train_df.columns:
    cols_to_drop.append('Id')

train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df = test_df.drop(columns=cols_to_drop, errors='ignore')

# 3.2 Tách biến
X = train_df.drop(columns=['Class'])
y_raw = train_df['Class']
X_test = test_df.copy()

# Xác định cột số và cột chữ
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# 3.3 Điền dữ liệu thiếu (Imputing)
# Số: điền trung vị (median) - tốt hơn mean khi có outliers
imputer_num = SimpleImputer(strategy='median')
X[numeric_cols] = imputer_num.fit_transform(X[numeric_cols])
X_test[numeric_cols] = imputer_num.transform(X_test[numeric_cols])

# Chữ: điền giá trị phổ biến nhất
imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = imputer_cat.fit_transform(X[categorical_cols])
X_test[categorical_cols] = imputer_cat.transform(X_test[categorical_cols])

# 3.4 Chuẩn hóa dữ liệu số (Scaling) -> Quan trọng cho một số thuật toán
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# 3.5 Mã hóa biến phân loại (Encoding)
# Label Encoding cho Artist Name
for col in categorical_cols:
    le = LabelEncoder()
    # Combine để học hết các label có thể có
    combined_data = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(combined_data)
    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

# 3.6 Mã hóa Target (Class)
target_le = LabelEncoder()
y = target_le.fit_transform(y_raw)

# ==============================================================================
# 4. XÂY DỰNG ENSEMBLE MODEL (KẾT HỢP SỨC MẠNH)
# ==============================================================================

# Cấu hình các mô hình con (Hyperparameters cơ bản đã được tối ưu nhẹ)
clf1 = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)

clf2 = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

clf3 = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    random_seed=42,
    verbose=0, # Tắt log in ra màn hình cho gọn
    allow_writing_files=False
)

# Tạo Voting Classifier (Soft Voting: tính xác suất trung bình)
voting_clf = VotingClassifier(
    estimators=[
        ('xgb', clf1),
        ('lgbm', clf2),
        ('cat', clf3)
    ],
    voting='soft'
)

# ==============================================================================
# 5. HUẤN LUYỆN VÀ ĐÁNH GIÁ (VALIDATION)
# ==============================================================================
# Chia tập validation để kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

print("Đang huấn luyện mô hình Ensemble (XGB + LGBM + CatBoost)...")
voting_clf.fit(X_train, y_train)

# Đánh giá
print("Đang đánh giá...")
y_pred_val = voting_clf.predict(X_val)
f1 = f1_score(y_val, y_pred_val, average='micro') # Micro-F1 thường dùng cho Multi-class imbalance

print(f"\n>>> F1-Score trên tập Validation: {f1:.5f}")
print("-" * 30)

# ==============================================================================
# 6. HUẤN LUYỆN LẠI TRÊN TOÀN BỘ DỮ LIỆU & TẠO FILE SUBMISSION (ĐÃ SỬA LỖI PATH)
# ==============================================================================
# Để đạt hiệu quả cao nhất, ta train lại mô hình trên TOÀN BỘ tập Train (gộp cả train + val)
print("Đang huấn luyện lại trên toàn bộ dữ liệu (Full Train)...")
voting_clf.fit(X, y)

print("Đang dự đoán tập Test...")
predictions = voting_clf.predict(X_test)

# Chuyển ngược từ số về tên Class (Rock, Pop...)
predictions_labels = target_le.inverse_transform(predictions)

# --- PHẦN SỬA LỖI QUAN TRỌNG ---
# Đọc sample submission để lấy Id chính xác (Đường dẫn cho Colab)
try:
    # Thử đọc file sample_submission.csv nếu bạn đã upload lên Colab
    sub = pd.read_csv('sample_submission (2).csv')
    sub['Class'] = predictions_labels
    sub.to_csv('submission_ensemble.csv', index=False)
except FileNotFoundError:
    print("Không tìm thấy file sample_submission.csv, đang tự tạo file mới...")
    # Nếu không có file mẫu, tự tạo dựa trên file test.csv
    # Lưu ý: File test.csv cũng phải được upload lên Colab rồi
    test_data_check = pd.read_csv('test (2).csv')

    # Tạo khung dataframe
    sub = pd.DataFrame()

    # Nếu trong test.csv có cột Id thì lấy, không thì dùng index + 1
    if 'Id' in test_data_check.columns:
        sub['Id'] = test_data_check['Id']
    else:
        sub['Id'] = test_data_check.index + 1

    sub['Class'] = predictions_labels
    sub.to_csv('submission_ensemble.csv', index=False)

print("\nĐã tạo file 'submission_ensemble.csv'.")
display(sub.head())

Train shape: (14396, 18)
Test shape: (3600, 17)
Đang huấn luyện mô hình Ensemble (XGB + LGBM + CatBoost)...
Đang đánh giá...

>>> F1-Score trên tập Validation: 0.53056
------------------------------
Đang huấn luyện lại trên toàn bộ dữ liệu (Full Train)...
Đang dự đoán tập Test...

Đã tạo file 'submission_ensemble.csv'.


,Id,Class
0,14397,9
1,14398,6
2,14399,9
3,14400,10
4,14401,5
